In [171]:
import json
from tqdm import tqdm

In [107]:
# Load evidence snippets
with open('/kaggle/input/quantemp-evidence-snippets/test/nli_input_test_reranktop5.json', 'r') as f:
    evidence_snippets = json.load(f)

with open('/kaggle/input/quantemp/test_claims_quantemp.json', 'r') as f:
    test_claims = json.load(f)
    
test_claims[0]
# evidence_corpus[0]

{'country_of_origin': 'usa',
 'label': 'Conflicting',
 'url': 'https://www.politifact.com/factchecks/2012/apr/17/roger-williams/roger-williams-says-cbo-forecasts-obamacare-will-c/',
 'lang': 'en',
 'claim': '"The non-partisan Congressional Budget Office concluded ObamaCare will cost the U.S. more than 800,000 jobs."',
 'doc': 'Republican U.S House candidate Roger Williams of Texas promises to "end the fraud of Obamacare" in a mailer to Austin-area voters we spotted April 11, 2012.His mailer lists among promises broken by President Obama the notion that the 2010 overhaul of health care laws would create jobs. Under "Promise Broken," the mailer says: "The non-partisan Congressional Budget Office concluded ObamaCare will cost the U.S. more than 800,000 jobs."Really?Williams’ footnoted backup was a Feb. 11, 2011, blog post on AmericanThinker.com saying that Doug Elmendorf, director of the Congressional Budget Office, had just testified to the House Budget Committee that as of 2020-21, the 

In [108]:
# Load the claims and questions
def split_questions(questions_list):
    questions_str = " ".join(questions_list)
    questions = [q.strip() for q in questions_str.split('?') if q.strip()]
    questions = [f"{q}?" for i, q in enumerate(questions) if not q.endswith('?')]
    return questions

with open('/kaggle/input/strategyqa-decomp-quantemp/strategyqa_decomp_test.json', 'r') as f:
    data = json.load(f)
    
    for i in range(len(data)):
        questions = split_questions(data[i]['decomposition'])
        data[i]['decomposition'] = questions
        data[i]['label'] = test_claims[i]['label']
        data[i]['taxonomy_label'] = test_claims[i]['taxonomy_label']


data[0]

{'claim': '"The non-partisan Congressional Budget Office concluded ObamaCare will cost the U.S. more than 800,000 jobs."',
 'decomposition': ['How much does ObamaCare cost the US?',
  "What is the non-partisan Congressional Budget Office's estimate of how much does Obamacare cost the U.S.?",
  'Is #1 greater than 800,000 jobs?'],
 'label': 'Conflicting',
 'taxonomy_label': 'statistical'}

In [109]:
for i in range(len(data)):
    data[i]['evidence'] =  evidence_snippets[i]['evidence']

data[1]

{'claim': '"More than 50 percent of immigrants from (El Salvador, Guatemala and Honduras) use at least one major welfare program once they get here."',
 'decomposition': ['What is the major welfare program that El Salvador, Guatemala and Honduras use?',
  'Is #1 included in #2?'],
 'label': 'True',
 'taxonomy_label': 'statistical',
 'evidence': 'welfare use is very high for all three top-sending countries from the region. of households headed by el salvadoran immigrants, 60 percent use at least one major welfare program; the figure is 59 percent for guatemalans and 52 percent for hondurans. given the large share of central americans with low incomes, it is not surprising that so many access the welfare system. in 2018, 56 percent of households headed by central american immigrants used one or more major welfare programs, more than double the 26 percent of native households. welfare use is common among both recently arrived and more established central americans. among households headed

In [110]:
def get_qa_context(data, previous_answers):
    questions = '\n'.join([f"[#{i+1}]: {q} {a}" for i, (q, a) in enumerate(zip(data['decomposition'], previous_answers))])
#     context = f"""[Claim]: {data['claim']}
    context = f"""[Facts]: {questions}
    [Evidence]: {data['evidence']}
    """
    return context

get_qa_context(data[0], ['yes'])


'[Facts]: [#1]: How much does ObamaCare cost the US? yes\n    [Evidence]: this non-partisan congressional budget office report confirms that obamacare will cost america 2 million jobs ... budget analysts said in the most detailed ... ... earlier this year, the head of the congressional budget office testified before congress that implementation of obamacare would cost 800,000 jobs.  ... jun 14, 2011  ... congressional budget office testified before congress that implementation of obamacare would cost 800,000 jobs." bachmann made nearly the ... the director of the congressional budget office testified last year that obamacare will destroy 800,000 jobs and this summer the u.s. chamber of commerce ... 2014-02-24  the non-partisan congressional budget office recently reported that obamacare will shrink the economy by the equivalent of 2.5 million full-time ...\n    '

In [111]:
print(len(data))
print(len(evidence_snippets))

2495
2495


In [112]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [122]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
model_name = "deepset/deberta-v3-base-squad2"
# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=device)
answers = []

csv_data = []
for d in tqdm(data):
    if d['taxonomy_label'] != 'comparison':
        continue
    
    csv = [d['claim'], d['evidence']]
    for question in d['decomposition']:
        QA_input = {
            'question': question,
            'context': get_qa_context(d,answers)
        }
        res = nlp(QA_input)
        csv += [question, f"{res['answer']} ({res['score']:.3f})"]

        answers.append(res['answer'])
    csv_data.append(csv)

100%|██████████| 2495/2495 [00:38<00:00, 65.26it/s] 


In [170]:
pd.set_option('display.max_rows', 100)
df

,0,1,2,3,4,5,6,7,8,9
0,"???And just like that, your $15/hr doesn???t b...","and just like that, your $15/hr doesn't buy yo...",How much does $15/hr pay?,$15/hr doesn't buy you as much (0.159),What is #1 multiplied by a year?,percentage of living wage (0.000),None,None,None,None
1,Two UK police forces used the same photo to il...,high contrast film was used to photograph the ...,What is the crime depicted in the photo above?,[#1]: What (0.015),Is #1 and #2 the same?,"different classes of crime, as well as of dif...",None,None,None,None
2,"As a sign of ""extraordinary progress,"" ""there ...",the report also highlights an alarming increas...,How many children are out of school in the US ...,258 million (0.102),What is the achievement rate for primary-age c...,percentage of living wage (0.004),Is #1 greater than or equal to 50%?,percentage of living wage (0.015),None,None
3,"""Congress and President Obama have accumulated...",president barack obama's administration racked...,How much debt has President Obama accumulated ...,$9.3 trillion (0.651),What is the total amount of debt in the United...,$10.6 trillion (0.680),None,None,None,None
4,"The American Health Care Act ""will reduce prem...",24 mar 2017 speaker paul ryan says the house ...,How much does the American Health Care Act cost?,$15/hr doesn't buy you as much (0.003),What is the average cost of insurance in 2018?,$15/hr doesn't buy you as much (0.005),None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
250,"""Gangs have increased by 40 percent since this...","2023-09-20 since president biden took office,...",How many gang members has President Donald Tru...,$15/hr doesn't buy you as much (0.001),What is the population of Chicago?,How many gang members has President Donald Tr...,Is #2 greater than or equal to #1?,\n[#2]: What is the population of Chicago? pe...,None,None
251,"Says ""6 percent more Republicans across the st...",its also important to put these numbers in con...,What percentage of votes did Republicans recei...,58 percent of the popular vote. (0.636),How many votes did Democrats receive in Michig...,58 percent of the popular vote. (0.100),Is #1 greater than #2?,the republican party was awarded 3 more total...,None,None
252,"""Since the end of 2012, South Africa's unemplo...",since the end of 2012 south africa's unemploye...,What is the unemployment rate in South Africa?,percentage of living wage (0.003),What are the unemployment rates in the Western...,percentage of living wage (0.057),None,None,None,None
253,"""In 1978, a student who worked a minimum-wage ...","the meme said that ""in 1978, a student who wor...",What was the minimum wage in 1978?,$2.65. (0.998),How much does a year of tuition cost?,$688 (0.970),None,None,None,None
